# Projeto Automação Web - Busca de Preços e Produtos

### Objetivo: 

- treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade") e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail tikogamer04@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [2]:
#criar o navegador (web drivrer)
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep




navegador = webdriver.Chrome()



In [14]:
#importar e e visualizar a base de dados 
produtos = pd.read_excel("buscas.xlsx")
display(produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 14 64gb,mini watch,4000,5500
1,Playstation 5,jogos,3500,4500


In [15]:

def busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo):
    #tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    Lista_termos_banidos = termos_banidos.split(" ")
    Lista_termos_nome_produto = produto.split(" ")
    lista_resultados = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    #entar no google
    navegador.get("https://www.google.com.br/")
    sleep(0.3)

    navegador.find_element(By.CLASS_NAME,"gLFyf").send_keys(produto,Keys.ENTER)
    sleep(0.3)

    #entar na aba do shopping
    elementos = navegador.find_elements(By.CLASS_NAME,"GKS7s")
    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break

    # pega as informçaões do produto
        
    lista_informacoes = navegador.find_elements(By.CLASS_NAME,"i0X6df")

    for informacao in lista_informacoes:
        nome = informacao.find_element(By.CLASS_NAME,"tAxDx").text
        nome = nome.lower()

        #analisar se não tem nenhum termo banido
        tem_termos_banidos = False
        for palavra in Lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos=True
       
        # analisar se tem todos termos do nome do produto
        tem_todos_termos_produtos = True
        for palavra in Lista_termos_nome_produto:
            if palavra not in nome:
                tem_todos_termos_produtos = False   
       
           
        # Selecionar apenas tem_termos_banidos = false e tem_todos_termos_produto = true
        if not tem_termos_banidos and tem_todos_termos_produtos == True:        
            preco = informacao.find_element(By.CLASS_NAME,"a8Pemb").text
            preco = preco.replace("R$","").replace(" ","").replace(".","").replace(",",".")
            preco = float(preco)

            # verificar se o preço esta entre o preco_minimo e o preco_maximo
          
            elemento_referencia = informacao.find_element(By.CLASS_NAME,"bONr3b")
            elemento_pai = elemento_referencia.find_element(By.XPATH,"..")
            link = elemento_pai.get_attribute("href")
            lista_resultados.append((nome,preco,link))    
            
    return lista_resultados






In [16]:
tabela_ofertas = pd.DataFrame()
for linha in produtos.index:
    #pesquisa pelo produto (produto da base de dados)
    produto = produtos.loc[linha,"Nome"]
    termos_banidos = produtos.loc[linha,"Termos banidos"]
    preco_minimo = produtos.loc[linha,"Preço mínimo"]
    preco_maximo =  produtos.loc[linha,"Preço máximo"]

    lista_busca_google_shoppinh = busca_google_shopping(navegador,produto,termos_banidos,preco_minimo,preco_maximo)
    if lista_busca_google_shoppinh:
        tabela_google_shopping = pd.DataFrame(lista_busca_google_shoppinh,columns=["produto","preço","link"])
        tabela_ofertas = pd.concat([tabela_ofertas,tabela_google_shopping])
display(tabela_ofertas)
   

print("-="*20)


,produto,preço,link
0,iphone apple 64gb 3gb ram c/ ios 14 tela 6 cam...,2535.28,https://www.google.com.br/url?url=http://loja....
0,playstation 5 sony,2856.49,https://www.google.com.br/url?url=https://www....
1,"console playstation 5 edição digital, ssd 825g...",4432.50,https://www.google.com.br/url?url=https://www....
2,console playstation 5 digital edition - sony,3899.90,https://www.google.com.br/url?url=https://www....
3,console playstation 5 + marvel's spider-man 2 ...,3672.57,https://www.google.com.br/url?url=https://www....
4,console playstation 5 825gb sony standard,3554.00,https://www.google.com.br/url?url=https://www....
5,"playstation 5 slim, digital edition, 1tb ssd, ...",3199.90,https://www.google.com.br/url?url=https://www....
6,console playstation 5 slim edição digital sony...,3799.00,https://www.google.com.br/url?url=https://www....
7,playstation console ps5 digital slim prateado ...,2637.31,https://www.google.com.br/url?url=https://shop...
8,"playstation 5, spider-man 2 bundle, com leitor...",3559.90,https://www.google.com.br/url?url=https://www....


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [7]:
# esportar para o excel 
tabela_ofertas.to_excel("Ofertas.xlsx")

